In [1]:
import os
os.chdir('/kaggle/input/myfunctions')
from functions import *
os.chdir('/kaggle/input/myfuncs')
from myutils import *
os.chdir('/kaggle/input/mydatasets2')

2024-03-05 20:39:28.140185: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 20:39:28.140288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 20:39:28.270211: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
!ls

training_set1.pkl  training_set11.pkl


In [3]:
num_features = 1 # Number of features in the data (Example: Low, High, etc)
num_stocks = 100 # The dataset is a list of 2000 stock dataframes. num_stocks is the number of stocks to consider for training (Matter of RAM capacity)
num_epochs = 20
batch_size = 512
units = 128
models_dict = {}
train_metrics_dict = {}
valid_metrics_dict = {}
test_metrics_dict = {}

In [4]:
time_steps = 20
x_train, x_valid, y_train, y_valid = preprocess_data_equal_division('training_set1.pkl',split = True, time_steps=time_steps-1, num_stocks = None,
                                            only_close=True,
                                            equal_split=False)

In [5]:
x_train = x_train.reshape((*x_train.shape, 1))

In [6]:
x_valid = x_valid.reshape((*x_valid.shape, 1))

In [7]:
model = Sequential((LSTM(3*units, input_shape=(time_steps+1,num_features), return_sequences=True),
                       LSTM(3*units, return_sequences=False),
                       Dense(4*units, activation='relu'),
                       Dense(4*units, activation='relu'),
#                        Dense(4*units, activation='relu'),
                       Dense(3),
                       Softmax()))
model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(learning_rate=0.001), metrics=['Accuracy'])
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 21, 384)        │       592,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 384)            │     1,181,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │         1,539 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Softmax)               │ (None, 3)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,235,395 (8.53 MB)

 Trainable params: 2,235,395 (8.53 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
callbacks = [EarlyStopping(patience=15, restore_best_weights=True, min_delta=1e-4), ReduceLROnPlateau(verbose=True)]

In [10]:
# model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=num_epochs, batch_size=batch_size)
# models_dict[time_steps] = model
# print(f"Time steps = {time_steps}")
# trained for 20 epochs and once again training for single epoch
model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=1, batch_size=batch_size)
models_dict[time_steps] = model
print(f"Time steps = {time_steps}")

6819/6819 ━━━━━━━━━━━━━━━━━━━━ 200s 29ms/step - Accuracy: 0.5396 - loss: 0.7780 - val_Accuracy: 0.5356 - val_loss: 0.7801
Time steps = 20


In [11]:
train_metrics_dict[time_steps] = metric_calculations_categorical(model, x_train, y_train, batch_size=8192, set_="Training set")
valid_metrics_dict[time_steps] = metric_calculations_categorical(model, x_valid, y_valid, batch_size=8192,set_="Validation set")

427/427 ━━━━━━━━━━━━━━━━━━━━ 44s 103ms/step
For Training set
  Accuracy: 54.27764092575619,
  Precision: 54.5726262726607,
  Recall: 53.85138843531948,
  Specificity: 54.70831205131409,
  F1score: 54.20960851264754
107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 104ms/step
For Validation set
  Accuracy: 53.555912007332715,
  Precision: 53.827586525491064,
  Recall: 53.143856640502065,
  Specificity: 53.971963692797964,
  F1score: 53.4835364832083


In [12]:
x_test, y_test = preprocess_data_equal_division('training_set11.pkl', split=False, time_steps=time_steps-1, num_stocks = None,
                                        only_close=True,
                                        equal_split=False)

In [13]:
test_metrics_dict[time_steps] = metric_calculations_categorical(model, x_test.reshape(*x_test.shape, 1), y_test, batch_size=8192, set_="Test set")

260/260 ━━━━━━━━━━━━━━━━━━━━ 27s 103ms/step
For Test set
  Accuracy: 49.41938250872467,
  Precision: 49.310853647304434,
  Recall: 48.432720631887236,
  Specificity: 50.40230332854255,
  F1score: 48.86784254478


In [19]:
from tensorflow.keras.models import clone_model
import pickle
lis = [train_metrics_dict, 
valid_metrics_dict, 
test_metrics_dict]
file_path = '/kaggle/working/lstm_only_close.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(lis, file)
model.save('/kaggle/working/model.h5')